In [11]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
from plotly.subplots import make_subplots
import plotly.graph_objs as go
import copy
import os
import torch
from PIL import Image
from PIL import Image, ImageDraw
from torch.utils.data import Dataset
import torchvision.transforms as transforms
from torch.utils.data import random_split
from torch.optim.lr_scheduler import ReduceLROnPlateau
import torch.nn as nn
from torchvision import utils
%matplotlib inline

In [12]:
path = "/kaggle/input/aml-challenge1"
import pandas as pd

labels_df = pd.read_csv(path+'/train.csv')  # Adjust filename
print(labels_df.head())

                                     id  has_cactus
0  0004be2cfeaba1c0361d39e2b000257b.jpg           1
1  000c8a36845c0208e833c79c1bffedd1.jpg           1
2  000d1e9a533f62e55c289303b072733d.jpg           1
3  0011485b40695e9138e92d0b3fb55128.jpg           1
4  0014d7a11e90b62848904c1418fc8cf2.jpg           1


In [13]:
print(labels_df.shape)
labels_df[labels_df.duplicated(keep=False)]

(17500, 2)


,id,has_cactus


In [14]:
labels_df['has_cactus'].value_counts()

has_cactus
1    13136
0     4364
Name: count, dtype: int64

In [15]:
import torch
torch.manual_seed(0)

from torch.utils.data import Dataset
import os
import pandas as pd
from PIL import Image
import torchvision.transforms as transforms

class pytorch_data(Dataset):
    
    def __init__(self, data_dir, transform, data_type="train"):
        # Get Image File Names
        cdm_data = os.path.join(data_dir, data_type)
        file_names = os.listdir(cdm_data)

        all_image_paths = [os.path.join(cdm_data, f) for f in file_names if f.lower().endswith(('.jpg', '.png', '.jpeg'))]

        print(f"Found {len(all_image_paths)} images in directory.")
        print(f"Sample filenames: {all_image_paths[:1]}")

        # Get Labels
        labels_data = os.path.join("/kaggle/input/aml-challenge1/", "train.csv")
        labels_df = pd.read_csv(labels_data)

        # Normalize index: remove extensions if present
        labels_df['id'] = labels_df['id'].apply(lambda x: os.path.splitext(str(x))[0])
        labels_df.set_index("id", inplace=True)

        print(f"Labels dataframe length: {len(labels_df)}")

        # Extract only valid images (that have a label)
        valid_filenames = []
        labels = []

        for f in all_image_paths:
            filename = os.path.basename(f)  # get only file name
            image_id = os.path.splitext(filename)[0]  # remove extension

            if image_id in labels_df.index:
                valid_filenames.append(f)
                labels.append(labels_df.loc[image_id].values[0])
            else:
                print(f"Warning: image '{filename}' has no matching label in train.csv")

        self.full_filenames = valid_filenames
        self.labels = labels
        self.transform = transform

        print(f"Valid image-label pairs: {len(self.full_filenames)}")
        print(f"First few labels: {self.labels[:5]}")
      
    def __len__(self):
        return len(self.full_filenames)
      
    def __getitem__(self, idx):
        if idx >= len(self.full_filenames):
            raise IndexError(f"Index {idx} out of bounds for dataset of length {len(self.full_filenames)}")

        image = Image.open(self.full_filenames[idx])
        image = self.transform(image)
        return image, self.labels[idx]

In [16]:
# define transformation that converts a PIL image into PyTorch tensors
import torchvision.transforms as transforms
data_transformer = transforms.Compose([transforms.ToTensor(),
                                       transforms.Resize((32,32))])

In [17]:
# Define an object of the custom dataset for the train folder.
data_dir = path+'/train/'
img_dataset = pytorch_data(data_dir, data_transformer, "train")

Found 17500 images in directory.
Sample filenames: ['/kaggle/input/aml-challenge1/train/train/5d3a7d32516a92cc0dc8c52af515eaa4.jpg']
Labels dataframe length: 17500
Valid image-label pairs: 17500
First few labels: [1, 0, 1, 1, 1]


In [18]:
# define transformation that converts a PIL image into PyTorch tensors
data_transformer = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize((32, 32))
])

In [19]:
# Test a sample
img, label = img_dataset[10]
print(img.shape, torch.min(img), torch.max(img))

torch.Size([3, 32, 32]) tensor(0.2667) tensor(0.8627)


**DATA AUGMENTATION CHE VA A RADDOPPIARE IL NUMERO DI SAMPLE DELLA CLASSE 0 AVENDO ALLA FINE CHE I SAMPLE DELLA CLASSE 0 SONO I 2/3 DEI SAMPLE DELLA CLASSE 1**

In [23]:
# Aumentiamo le immagini della classe 0
from torchvision.transforms import RandomRotation, ToTensor, Resize
from tqdm import tqdm

# Trasformazione per augmentare
augment_transform = transforms.Compose([
    RandomRotation(degrees=10),
    Resize((32, 32)),
    ToTensor()
])

# Trova solo immagini con etichetta 0
images_class0 = [i for i in range(len(img_dataset)) if img_dataset.labels[i] == 0]

# Duplichiamo queste immagini con trasformazione
augmented_images = []
augmented_labels = []

for idx in tqdm(images_class0):
    img_path = img_dataset.full_filenames[idx]
    img = Image.open(img_path)
    augmented_img = augment_transform(img)
    augmented_images.append(augmented_img)
    augmented_labels.append(0)

# Stack immagini originali
original_images = [img_dataset[i][0] for i in range(len(img_dataset))]
original_labels = [img_dataset[i][1] for i in range(len(img_dataset))]

# Combina immagini originali + augmentate
all_images = torch.stack(original_images + augmented_images)
all_labels = torch.tensor(original_labels + augmented_labels)

# Nuovo Dataset custom con dati augmentati
class AugmentedDataset(torch.utils.data.Dataset):
    def __init__(self, images, labels):
        self.images = images
        self.labels = labels
    def __len__(self):
        return len(self.images)
    def __getitem__(self, idx):
        return self.images[idx], self.labels[idx]

# Sostituisci img_dataset con quello nuovo
img_dataset = AugmentedDataset(all_images, all_labels)

100%|██████████| 4364/4364 [00:10<00:00, 399.11it/s]


***DIVISIONE NEI TRE SET TRAIN, VALIDATION, TEST DOPO AVER AUMENTATO I DATI DELLA CLASSE 0***

In [45]:
from sklearn.model_selection import train_test_split
from torch.utils.data import Subset

# Estrai le etichette in modo sicuro
labels = img_dataset.labels  # NON usare img_dataset[i][1]

# Crea una lista di tutti gli indici
all_indices = list(range(len(img_dataset)))

# Split stratificato: Train (70%) e Temp (30%)
train_idx, temp_idx = train_test_split(
    all_indices, test_size=0.3, stratify=labels, random_state=42
)

# Estrai le label corrispondenti agli indici temporanei per secondo split
temp_labels = [labels[i] for i in temp_idx]

# Split stratificato: Validation (15%) e Test (15%) da temp
val_idx, test_idx = train_test_split(
    temp_idx, test_size=0.5, stratify=temp_labels, random_state=42
)

# Crea i subset PyTorch
train_ts = Subset(img_dataset, train_idx)
val_ts = Subset(img_dataset, val_idx)
test_ts = Subset(img_dataset, test_idx)

# Visualizzazione
print("train dataset size:", len(train_ts))
print("validation dataset size:", len(val_ts))
print("test dataset size:", len(test_ts))

train dataset size: 15304
validation dataset size: 3280
test dataset size: 3280


**DATA LOADER FOR SPLITTING IMAGES**

In [51]:
from torch.utils.data import DataLoader

# Training DataLoader
train_dl = DataLoader(train_ts,
                      batch_size=64, 
                      shuffle=True)

# Validation DataLoader
val_dl = DataLoader(val_ts,
                    batch_size=64,
                    shuffle=False)

**DEFINIZIONE DEL MODELLO CNN**

In [52]:
def findConv2dOutShape(hin,win,conv,pool=2):
    # get conv arguments
    kernel_size=conv.kernel_size
    stride=conv.stride
    padding=conv.padding
    dilation=conv.dilation

    hout=np.floor((hin+2*padding[0]-dilation[0]*(kernel_size[0]-1)-1)/stride[0]+1)
    wout=np.floor((win+2*padding[1]-dilation[1]*(kernel_size[1]-1)-1)/stride[1]+1)

    if pool:
        hout/=pool
        wout/=pool
    return int(hout),int(wout)

import torch.nn as nn
import torch.nn.functional as F

# Neural Network
class Network(nn.Module):
    
    # Network Initialisation
    def __init__(self, params):
        
        super(Network, self).__init__()
    
        Cin,Hin,Win=params["shape_in"]
        init_f=params["initial_filters"] 
        num_fc1=params["num_fc1"]  
        num_classes=params["num_classes"] 
        self.dropout_rate=params["dropout_rate"] 
        
        # Convolution Layers
        self.conv1 = nn.Conv2d(Cin, init_f, kernel_size=3, padding=1)
        h,w=findConv2dOutShape(Hin,Win,self.conv1)
        self.conv2 = nn.Conv2d(init_f, 2*init_f, kernel_size=3,padding=1)
        h,w=findConv2dOutShape(h,w,self.conv2)
        self.conv3 = nn.Conv2d(2*init_f, 4*init_f, kernel_size=3, padding=1)
        h,w=findConv2dOutShape(h,w,self.conv3)
        
        # compute the flatten size
        self.num_flatten=h*w*4*init_f   #4*8 -> 32
        self.fc1 = nn.Linear(self.num_flatten, num_fc1)
        self.fc2 = nn.Linear(num_fc1, num_classes)

    

    def forward(self,X):
        # Convolution & Pool Layers
        X = F.relu(self.conv1(X));
        X = F.max_pool2d(X, 2, 2)
        X = F.relu(self.conv2(X))
        X = F.max_pool2d(X, 2, 2)
        X = F.relu(self.conv3(X))
        X = F.max_pool2d(X, 2, 2)

        b = X.shape[0]
        X = X.view(b,-1) #torch appiattisce su tutti i canali, tranne il primo che corrisponde alla batch size
        
        X = F.relu(self.fc1(X))
        X=F.dropout(X, self.dropout_rate)
        X = self.fc2(X)
        return F.log_softmax(X, dim=1)

**PARAMETRI DEL MODELLO**

In [53]:
# Neural Network Predefined Parameters
params_model={
        "shape_in": (3,32,32), 
        "initial_filters": 8,    
        "num_fc1": 100,
        "dropout_rate": 0.25,
        "num_classes": 2}

# Create instantiation of Network class
cnn_model = Network(params_model)

# define computation hardware approach (GPU/CPU)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = cnn_model.to(device)

**GRID SEARCH PER IL TUNING DEI PARAMETRI**

In [57]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from sklearn.metrics import accuracy_score

# 🔧 Hyperparameter grid
lr_list = [0.001, 0.005, 0.01]
dropout_list = [0.25, 0.4, 0.5]
filters_list = [8, 16, 32]
fc1_list = [64, 100, 128]

# 📦 Funzione per addestramento + validazione
def train_and_evaluate(model, train_dl, val_dl, lr, device, epochs=5):
    model = model.to(device)
    optimizer = optim.Adam(model.parameters(), lr=lr)
    criterion = nn.NLLLoss()

    model.train()
    for epoch in range(epochs):
        for X_batch, y_batch in train_dl:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            optimizer.zero_grad()
            output = model(X_batch)
            loss = criterion(output, y_batch)
            loss.backward()
            optimizer.step()

    model.eval()
    y_true, y_pred = [], []
    with torch.no_grad():
        for X_batch, y_batch in val_dl:
            X_batch = X_batch.to(device)
            output = model(X_batch)
            preds = output.argmax(dim=1).cpu().numpy()
            y_pred.extend(preds)
            y_true.extend(y_batch.numpy())

    acc = accuracy_score(y_true, y_pred)
    return acc

# 🚀 Grid Search
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
best_acc = 0
best_params = {}

for lr in lr_list:
    for dropout in dropout_list:
        for filters in filters_list:
            for fc1 in fc1_list:
                params_model = {
                    "shape_in": (3, 32, 32),
                    "initial_filters": filters,
                    "num_fc1": fc1,
                    "dropout_rate": dropout,
                    "num_classes": 2
                }

                model = Network(params_model)
                acc = train_and_evaluate(model, train_dl, val_dl, lr, device, epochs=5)

                print(f"LR: {lr}, Dropout: {dropout}, Filters: {filters}, FC1: {fc1} -> Val Accuracy: {acc:.4f}")

                if acc > best_acc:
                    best_acc = acc
                    best_params = {
                        "lr": lr,
                        "dropout_rate": dropout,
                        "initial_filters": filters,
                        "num_fc1": fc1
                    }

print(f"🏆 Best Accuracy: {best_acc:.4f} with params {best_params}")

LR: 0.001, Dropout: 0.25, Filters: 8, FC1: 64 -> Val Accuracy: 0.9637
LR: 0.001, Dropout: 0.25, Filters: 8, FC1: 100 -> Val Accuracy: 0.9387
LR: 0.001, Dropout: 0.25, Filters: 8, FC1: 128 -> Val Accuracy: 0.9619
LR: 0.001, Dropout: 0.25, Filters: 16, FC1: 64 -> Val Accuracy: 0.9695
LR: 0.001, Dropout: 0.25, Filters: 16, FC1: 100 -> Val Accuracy: 0.9793
LR: 0.001, Dropout: 0.25, Filters: 16, FC1: 128 -> Val Accuracy: 0.9689
LR: 0.001, Dropout: 0.25, Filters: 32, FC1: 64 -> Val Accuracy: 0.9823
LR: 0.001, Dropout: 0.25, Filters: 32, FC1: 100 -> Val Accuracy: 0.9811
LR: 0.001, Dropout: 0.25, Filters: 32, FC1: 128 -> Val Accuracy: 0.9817
LR: 0.001, Dropout: 0.4, Filters: 8, FC1: 64 -> Val Accuracy: 0.9610
LR: 0.001, Dropout: 0.4, Filters: 8, FC1: 100 -> Val Accuracy: 0.9622
LR: 0.001, Dropout: 0.4, Filters: 8, FC1: 128 -> Val Accuracy: 0.9649
LR: 0.001, Dropout: 0.4, Filters: 16, FC1: 64 -> Val Accuracy: 0.9540
LR: 0.001, Dropout: 0.4, Filters: 16, FC1: 100 -> Val Accuracy: 0.9726
LR: 0.00

**USO I BEST IPERPARAMETRI SUL TEST**

🏆 Best Accuracy: 0.9823 with params {'lr': 0.001, 'dropout_rate': 0.25, 'initial_filters': 32, 'num_fc1': 64}

In [58]:
best_params = {
    'lr': 0.001,
    'dropout_rate': 0.25,
    'initial_filters': 32,
    'num_fc1': 64
}

# Costruzione del modello finale
final_params = {
    "shape_in": (3, 32, 32),
    "initial_filters": best_params["initial_filters"],
    "num_fc1": best_params["num_fc1"],
    "dropout_rate": best_params["dropout_rate"],
    "num_classes": 2
}

final_model = Network(final_params).to(device)

In [59]:
from torch.utils.data import ConcatDataset, DataLoader

# Combina train + validation set
full_train_set = ConcatDataset([train_ts, val_ts])
full_train_loader = DataLoader(full_train_set, batch_size=64, shuffle=True)

# Ottimizzatore e loss
optimizer = torch.optim.Adam(final_model.parameters(), lr=best_params["lr"])
criterion = nn.NLLLoss()

# Training
final_model.train()
for epoch in range(10):
    running_loss = 0.0
    for X_batch, y_batch in full_train_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        optimizer.zero_grad()
        output = final_model(X_batch)
        loss = criterion(output, y_batch)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f"Epoch {epoch+1} - Loss: {running_loss:.4f}")

Epoch 1 - Loss: 65.1843
Epoch 2 - Loss: 25.6979
Epoch 3 - Loss: 17.9817
Epoch 4 - Loss: 16.0125
Epoch 5 - Loss: 12.0859
Epoch 6 - Loss: 10.3995
Epoch 7 - Loss: 7.9387
Epoch 8 - Loss: 7.8818
Epoch 9 - Loss: 7.4810
Epoch 10 - Loss: 5.5941


In [60]:
test_loader = DataLoader(test_ts, batch_size=64, shuffle=False)

# Valutazione
final_model.eval()
y_true, y_pred = [], []

with torch.no_grad():
    for X_batch, y_batch in test_loader:
        X_batch = X_batch.to(device)
        output = final_model(X_batch)
        preds = output.argmax(dim=1).cpu().numpy()
        y_pred.extend(preds)
        y_true.extend(y_batch.numpy())

from sklearn.metrics import accuracy_score, classification_report

print("🎯 Test Accuracy:", accuracy_score(y_true, y_pred))
print(classification_report(y_true, y_pred, digits=4))

🎯 Test Accuracy: 0.9896341463414634
              precision    recall  f1-score   support

           0     0.9804    0.9939    0.9871      1309
           1     0.9959    0.9868    0.9913      1971

    accuracy                         0.9896      3280
   macro avg     0.9882    0.9903    0.9892      3280
weighted avg     0.9897    0.9896    0.9896      3280



In [ ]:
# Percorso delle immagini di test
test_path = "/kaggle/input/aml-challenge1/test/test"
test_image_paths = [os.path.join(test_path, f) for f in os.listdir(test_path) if f.endswith(".jpg")]

# Trasformazioni da usare (uguali al training)
transform = transforms.Compose([
    transforms.Resize((32, 32)),   # Adatta alla dimensione usata in training
    transforms.ToTensor()
])

# Esegui il modello sul test set
model.eval()
predictions = []

with torch.no_grad():
    for path in test_image_paths:
        image = Image.open(path).convert("RGB")
        image = transform(image).unsqueeze(0)  # Add batch dimension

        output = model(image)
        pred = torch.argmax(output, dim=1).item()

        filename = os.path.basename(path)
        predictions.append((filename, pred))

# Salva in CSV
submission_df = pd.DataFrame(predictions, columns=["id", "has_cactus"])
submission_df.to_csv("submission.csv", index=False)
submission_df.head()

In [ ]:
import pandas as pd

# Leggi il file CSV con le predizioni
submission_df = pd.read_csv("submission.csv")

# Conta le occorrenze di ciascuna classe
class_counts = submission_df["has_cactus"].value_counts()

# Stampa i risultati
print("Conteggio predizioni per classe:")
print(class_counts)